## There's a little magic in everything

In [1]:
obj = object()

That called `object.__init__`

In [2]:
obj

That called `obj.__repr__`

In [3]:
str(obj)

'<object object at 0x7fbc320ffc70>'

That called `obj.__str__` (which calls `obj.__repr__` as a fallback)

```
















```

## Stringy Types

Implementing [`__repr__`](https://docs.python.org/3/reference/datamodel.html#object.__repr__) and [`__str__`](https://docs.python.org/3/reference/datamodel.html#object.__str__) can often be useful when debugging or manipulating objects interactively (like via Notebooks or REPLs)

In [4]:
class Stringy:
    def __init__(self, name):
        self.name = name

    def __repr__(self) -> str:
        """
        ...If at all possible, this should look like a valid Python expression 
        that could be used to recreate an object with the same value. If this 
        is not possible, a string of the form <...some useful description...> 
        should be returned. 
        """
        return f"{self.__class__.__name__}('{self.name}')"
    
    def __str__(self) -> str:
        """        
        This method differs from `__repr__()` in that there is no expectation 
        that `__str__()` returns a valid Python expression: a more convenient 
        or concise representation can be used.
        """
        return f"Name: {self.name}"

In [5]:
a = Stringy('apple')
a

Stringy('apple')

In [6]:
repr(a)

"Stringy('apple')"

In [7]:
str(a)

'Name: apple'

---

Why not something like:

In [8]:
class StringyToo:
    def __init__(self, name):
        self.name = name
        
    def to_string(self):
        return f"Name: {self.name}"

In [9]:
b = StringyToo('cheese')
b

In [10]:
b.to_string()

'Name: cheese'

```
















```

### Quacks like...

String-like objects can be used in place of other strings

In [11]:
f"I have an object. {b}."

'I have an object. <__main__.StringyToo object at 0x7fbc250bfc50>.'

In [12]:
f"I have an object. {a}. It was made with: {a!r}"

"I have an object. Name: apple. It was made with: Stringy('apple')"

```
















```

In [13]:
def verbose(method):
    """Wrapper to print the method name, arguments, and result of a call"""
    def _method(self, *args):
        arg_str = ', '.join(repr(a) for a in args)
        result = method(self, *args)
        print(f'call: {self!r}.{method.__name__}({arg_str}) => {result!r}')
        return result
    return _method

## Math-able Types

Implementing the mathematical magic methods can let you use any of the infix operators to manipulate your objects as makes sense.

    x + y     __add__             x & y   __and__
    x - y     __sub__             x | y   __or__      
    x * y     __mul__             x ^ y   __xor__
    x / y     __truediv__         x << y  __lshift__
    x // y    __floordiv__        x >> y  __rshift__
    x ** y    __pow__
    x % y     __mod__
    x @ y     __matmul__

In [14]:
class DigitBase:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"{self.__class__.__name__}({self.value})"


class DigitA(DigitBase):
    @verbose
    def __add__(self, other) -> int:
        return self.value * 10 + other.value

In [15]:
DigitA(8) + DigitA(4)

call: DigitA(8).__add__(DigitA(4)) => 84


84

In [16]:
DigitA(8).__add__(DigitA(4))

call: DigitA(8).__add__(DigitA(4)) => 84


84

---

In [17]:
class DigitB(DigitBase):
    @verbose
    def __add__(self, other) -> int:
        if isinstance(other, self.__class__):
            other_value = other.value
        else:
            other_value = other

        return self.value * 10 + other_value

In [18]:
DigitB(8) + 6

call: DigitB(8).__add__(6) => 86


86

In [19]:
# doesn't work
6 + DigitB(8)

TypeError: unsupported operand type(s) for +: 'int' and 'DigitB'

In [20]:
# also doesn't work (same error)
result = DigitB(1) + DigitB(2) + DigitB(3)

# equivalent
_intermediate: int = DigitB(1) + DigitB(2)
result = _intermediate + DigitB(3)

call: DigitB(1).__add__(DigitB(2)) => 12


TypeError: unsupported operand type(s) for +: 'int' and 'DigitB'

---

To support those operand type(s), we use the [*reflected* methods](https://docs.python.org/3/reference/datamodel.html?highlight=radd#object.__radd__), `__r*__`. When the "forward" direction fails, Python will attempt to call it the other way, so `6 + DigitB(8)` is effectively something like:

```python
try:
    6.__add__(DigitB(8))   # the int type doesn't know what a Digit is
except TypeError:
    DigitB(8).__radd__(6)  # but we have the option to do something!
```

In [21]:
class DigitC(DigitB):
    # __add__ from DigitB

    @verbose
    def __radd__(self, other) -> int:
        """
        Note that the order of arguments are reversed on the call; 
        so `self` will be the object *after* the infix operator, and 
        `other` the one before.
        
        We're also assuming that other is an integer, Digit + Digit
        operations will just use __add__ from the first one.
        """
        return other * 10 + self.value

In [22]:
2 + DigitC(7)

call: DigitC(7).__radd__(2) => 27


27

In [23]:
DigitC(1) + DigitC(2)

call: DigitC(1).__add__(DigitC(2)) => 12


12

In [24]:
DigitC(1) + DigitC(2) + DigitC(3) + DigitC(4) + DigitC(5)

call: DigitC(1).__add__(DigitC(2)) => 12
call: DigitC(3).__radd__(12) => 123
call: DigitC(4).__radd__(123) => 1234
call: DigitC(5).__radd__(1234) => 12345


12345

### Builtins that Add

The built-in `sum()` uses `+` to add objects together. It uses an optional keyword argument `start` as the initial value (all calls below will just use `__radd__`)

In [25]:
help(sum)

Help on built-in function sum in module builtins:

sum(iterable, start=0, /)
    Return the sum of a 'start' value (default: 0) plus an iterable of numbers
    
    When the iterable is empty, return the start value.
    This function is intended specifically for use with numeric values and may
    reject non-numeric types.



In [26]:
def complicated_str_to_int(s):
    digits = [DigitC(int(d)) for d in s]
    print(digits)
    print()
    return sum(digits)

assert complicated_str_to_int('8675309') == 8675309

[DigitC(8), DigitC(6), DigitC(7), DigitC(5), DigitC(3), DigitC(0), DigitC(9)]

call: DigitC(8).__radd__(0) => 8
call: DigitC(6).__radd__(8) => 86
call: DigitC(7).__radd__(86) => 867
call: DigitC(5).__radd__(867) => 8675
call: DigitC(3).__radd__(8675) => 86753
call: DigitC(0).__radd__(86753) => 867530
call: DigitC(9).__radd__(867530) => 8675309


```
















```

## Orderable Types

In [27]:
versions = [
    '0.9.4',
    '0.95.3',
    '0.10.5',
    '0.9.34',
    '0.10.42',
    '1.4.2',
]

In [28]:
sorted(versions)

['0.10.42', '0.10.5', '0.9.34', '0.9.4', '0.95.3', '1.4.2']

How can we sort these? We could pass a `key` argument into `sorted()`:

In [29]:
sorted(versions, key=lambda i: [int(p) for p in i.split('.')])

['0.9.4', '0.9.34', '0.10.5', '0.10.42', '0.95.3', '1.4.2']

In [30]:
class Version:
    def __init__(self, version):
        self.version = version
        self.major, self.minor, self.patch = (int(x) for x in version.split('.'))
    
    def __repr__(self):
        return f'{self.__class__.__name__}({self.version!r})'
    
    def __lt__(self, other):
#         print(f'call: __lt__({self}, {other})')
        return (
            (self.major, self.minor, self.patch) 
            < (other.major, other.minor, other.patch)
        )

In [31]:
Version('0.32.52') < Version('0.93.2')

True

In [32]:
Version('10.0.0') < Version('9.99.999')

False

In [33]:
version_objs = [Version(v) for v in versions]
version_objs

[Version('0.9.4'),
 Version('0.95.3'),
 Version('0.10.5'),
 Version('0.9.34'),
 Version('0.10.42'),
 Version('1.4.2')]

In [34]:
sorted(version_objs)

[Version('0.9.4'),
 Version('0.9.34'),
 Version('0.10.5'),
 Version('0.10.42'),
 Version('0.95.3'),
 Version('1.4.2')]

In [35]:
max(version_objs)

Version('1.4.2')

In [36]:
min(version_objs)

Version('0.9.4')

Also, not everything can take a `key` argument:

In [37]:
import heapq

version_heap = version_objs[:]
heapq.heapify(version_heap)

In [38]:
while version_heap:
    print(heapq.heappop(version_heap))

Version('0.9.4')
Version('0.9.34')
Version('0.10.5')
Version('0.10.42')
Version('0.95.3')
Version('1.4.2')


```
















```

## Container Methods

In [39]:
from contextlib import suppress

class FallbackDict:
    """Look through a series of dicts in order"""
    def __init__(self, *dicts):
        self.dicts = dicts

    def __getitem__(self, key):
        for d in self.dicts:
            with suppress(KeyError):  
                return d[key]
        raise KeyError(key)

In [40]:
dicts = [
    {"name": "chipy", "misc": "woot"},
    {"misc": "etc"},
    {"name": "unkown", "greeting": "hello"},
]
fbd = FallbackDict(*dicts)

f'{fbd["greeting"]}, {fbd["name"]}!'

'hello, chipy!'

It's kinda like a dictionary, we can index into it and get items, but not much else.

In [41]:
len(fbd)  # probably should be 3?

TypeError: object of type 'FallbackDict' has no len()

In [42]:
[k for k in fbd.values()]  # should be 'woot', 'hello', 'chipy'

AttributeError: 'FallbackDict' object has no attribute 'values'

In [43]:
'name' in fbd  # is a key in it?

KeyError: 0

In [44]:
# would be nice to convert to a normal dict if we wanted to JSON serialize it
dict(fbd)

KeyError: 0

### [Abstract Base Classes for Containers](https://docs.python.org/3/library/collections.abc.html) (`collections.abc`)

Amplify your magics! If you implement the abstract methods required by the type, it can provide additional mixin methods for free. Lets make our `FallbackDict` more `dict`-like by using the `Mapping` ABC (read-only dict)

In [45]:
import collections.abc


class FallbackDictM(FallbackDict, collections.abc.Mapping):
    # __init__, __getitem__ from FallbackDict
    
    def _all_keys(self):
        all_keys = set()
        for d in self.dicts:
            all_keys |= set(d)
        return all_keys
    
    def __len__(self):
        return len(self._all_keys())

    def __iter__(self):
        for key in self._all_keys():
            yield key

In [46]:
fbdm = FallbackDictM(*dicts)

In [47]:
print(len(fbdm))  # directly hitting __len__
for key in fbdm:  # hits __iter__
    print(key, fbdm[key])

3
name chipy
misc woot
greeting hello


In [48]:
list(fbdm.values())  # values method for free

['chipy', 'woot', 'hello']

In [49]:
'name' in fbdm  # __contains__ for free

True

In [50]:
flattened = dict(fbdm)
flattened

{'greeting': 'hello', 'misc': 'woot', 'name': 'chipy'}

In [51]:
flattened == fbdm  # __eq__ for free

True

```
















```

In [52]:
import re
import requests

def get_xkcd_hotlink(comic_number):
    """Extract the embedded URL for an XKCD comic (no magic here)"""
    match = re.search(
        r"Image URL \(for hotlinking/embedding\):\s*(?P<url>http.*(jpg|png))", 
        requests.get(f"https://xkcd.com/{comic_number}/").text,
    )
    if match:
        return match.groupdict()["url"]

## Proprietary Magic

https://ipython.readthedocs.io/en/stable/config/integrating.html#rich-display

IPython notebook can display richer representations of objects. To use this, you can define any of a number of `_repr_*_()` methods. Note that these are surrounded by single, not double underscores.

The notebook can display `svg`, `png`, `jpeg`, `html`, `javascript`, `markdown` and `latex`. If the methods don’t exist, or return `None`, it falls back to a standard `repr()`.

In [53]:
class XKCDViewer:        
    def __getitem__(self, key):
        url = get_xkcd_hotlink(key)
        if url is None:
            raise KeyError("couldn't find XKCD comic with that number")
        return XKCDComic(url)


class XKCDComic:
    def __init__(self, url):
        self.url = url

    def _repr_html_(self):
        return f"<img src='{self.url}'>"


xkcd = XKCDViewer()

In [55]:
ANTIGRAVITY = 353
BOBBY_TABLES = 327
PYTHON_ENV = 1987
SANDWICH = 149

xkcd[ANTIGRAVITY]

```
















```

## Bonus: Context Managers

In [56]:
import traceback

class expect_exception:
    """
    Expect an exception and trap it. Helps run all of a notebook
    that may have intentional errors for pedagogical reasons
    """
    def __init__(self, *exc_types):
        self.exc_types = exc_types
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, tb):
        if exc_type is None:
            raise RuntimeError('expected exception did not occur!')
        if issubclass(exc_type, self.exc_types):
            traceback.print_exception(exc_type, exc_value, tb)
            return True

In [57]:
with expect_exception(Exception):
    DigitA(3) + 3

Traceback (most recent call last):
  File "<ipython-input-57-2d1a4b0a29e5>", line 2, in <module>
    DigitA(3) + 3
  File "<ipython-input-13-94ae7d87db55>", line 5, in _method
    result = method(self, *args)
  File "<ipython-input-14-1411024cd395>", line 12, in __add__
    return self.value * 10 + other.value
AttributeError: 'int' object has no attribute 'value'


In [58]:
class Suppressor:
    """
    https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers
    """
    def __init__(self, exc_ignore):
        self.exc_ignore = exc_ignore

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        """
        ... If an exception is supplied, and the method wishes to suppress the 
        exception, it should return a true value. ...
        """
        return (
            exc_type is not None 
            and issubclass(exc_type, self.exc_ignore)
        )

In [59]:
with Suppressor(IndexError):
    print('abcde'[:3])  # works normally

with Suppressor(IndexError):
    print('abcde'[1000])  # raises an error, but it gets suppressed
    
print('mmmkay')

abc
mmmkay


In [60]:
with Suppressor(RuntimeError):
    'abcde'[1000]  # raises the wrong error

IndexError: string index out of range

We basically re-implemented [`contextlib.suppress`](https://docs.python.org/3/library/contextlib.html#contextlib.suppress) [(source)](https://github.com/python/cpython/blob/cca4eec3c0/Lib/contextlib.py#L342) from the standard lib.